In [1]:
import pandas as pd
import json
from itertools import accumulate
from tqdm import tqdm
from utils import get_response_openai

In [2]:
train = pd.read_parquet("../original_data/train.parquet")

In [ ]:
train["language"].value_counts()[:20]

In [4]:
language_ratios = train["language"].value_counts(normalize=True).to_dict()

with open("language_ratios.json", "w", encoding="utf-8") as f:
    json.dump(language_ratios, f, ensure_ascii=False, indent=4)

In [5]:
lmarena_ai_arena_human_preference_55k = pd.read_parquet("../external_data/lmarena-ai-arena-human-preference-55k.parquet")

In [ ]:
languages = []
translation_count = []

for language, ratio in language_ratios.items():
    if language != "English" and language != "unknown" and language != "xx" and language != "zzp":
        languages.append(language)
        translation_count.append(int(len(lmarena_ai_arena_human_preference_55k) * ratio) + 2)

print(languages)
print(translation_count)

In [7]:
prefix_count_sums = list(accumulate(translation_count))
# prefix_count_sums

In [ ]:
# lmarena_ai_arena_human_preference_55k = lmarena_ai_arena_human_preference_55k.sample(random_state=3407, frac=1.0)
lmarena_ai_arena_human_preference_55k.head()

In [ ]:
translation_aug_lmarena_ai_arena_human_preference_55k = pd.DataFrame(columns=lmarena_ai_arena_human_preference_55k.columns)

for idx, row in tqdm(lmarena_ai_arena_human_preference_55k.iterrows(), total=len(lmarena_ai_arena_human_preference_55k)):
    for i in range(len(prefix_count_sums)):
        if idx < prefix_count_sums[i] and i == 0:
            target_language = languages[i]
            break
        elif idx >= prefix_count_sums[i - 1] and idx < prefix_count_sums[i]:
            target_language = languages[i]
            break
        elif idx > prefix_count_sums[i] and i == len(prefix_count_sums) - 1:
            target_language = "English"
            break

    if target_language == "English":
        new_row = pd.DataFrame([{
            "id": row["id"],
            "prompt": row["prompt"],
            "response_a": row["response_a"],
            "response_b": row["response_b"],
            "winner": row["winner"],
            "model_a": row["model_a"],
            "model_b": row["model_b"],
            "language": row["language"]
        }])
        translation_aug_lmarena_ai_arena_human_preference_55k = pd.concat([translation_aug_lmarena_ai_arena_human_preference_55k, new_row], ignore_index=True)
    else:
        try:
            translated_prompt = get_response_openai(model="gpt-4o-mini", prompt=f"Translate the following text to {target_language}:\n{row['prompt']}")
            translated_response_a = get_response_openai(model="gpt-4o-mini", prompt=f"Translate the following text to {target_language}:\n{row['response_a']}")
            translated_response_b = get_response_openai(model="gpt-4o-mini", prompt=f"Translate the following text to {target_language}:\n{row['response_b']}")
            new_row = pd.DataFrame([{
                "id": row["id"],
                "prompt": translated_prompt,
                "response_a": translated_response_a,
                "response_b": translated_response_b,
                "winner": row["winner"],
                "model_a": row["model_a"],
                "model_b": row["model_b"],
                "language": target_language
            }])
            translation_aug_lmarena_ai_arena_human_preference_55k = pd.concat([translation_aug_lmarena_ai_arena_human_preference_55k, new_row], ignore_index=True)
        except:
            new_row = pd.DataFrame([{
                "id": row["id"],
                "prompt": row["prompt"],
                "response_a": row["response_a"],
                "response_b": row["response_b"],
                "winner": row["winner"],
                "model_a": row["model_a"],
                "model_b": row["model_b"],
                "language": row["language"]
            }])
            translation_aug_lmarena_ai_arena_human_preference_55k = pd.concat([translation_aug_lmarena_ai_arena_human_preference_55k, new_row], ignore_index=True)
            print(f"Error Index: {idx}")

In [ ]:
translation_aug_lmarena_ai_arena_human_preference_55k.to_csv("translation_aug_ratio_lmarena-ai-arena-human-preference-55k.csv", index=False)